# Advent of Code 2022 - Day 7

## Part One

In [1]:
from pathlib import Path
from typing import List

def read_input(input_name: str) -> List[str]:
    input_path = Path.cwd() / "input" / input_name

    with open(input_path, 'r') as input_file:
        
        lines = input_file.readlines()
        
    return [line.rstrip("\n") for line in lines]

In [2]:
terminal_output_test = read_input("day7_input_test.txt")
terminal_output = read_input("day7_input.txt")

Actions:

- $ cd / --> Start(Top Level)
- $ ls --> Items up to next '$' are either directories or files
- dir X --> create a FileDir of that nam
- 12345 X.something --> Create AFile for that object
- cd X --> Set current dir to FileDir of that name
- cd .. --> Set current dir to parent of currently open Dir


In [3]:
COMMAND = 1
ARGUMENT = 2

In [4]:
from dataclasses import dataclass

@dataclass
class AFile:
    
    name : str
    size : int

In [21]:
from __future__ import annotations

class FileDir:
    
    def __init__(self, dir_name: str, parent: str) -> None:
        self.dir_name = dir_name
        self.parent = parent
        self.contents = []
        
    def add_contents(self, new_content: AFile | FileDir ):
        
        self.contents.append(new_content)
    
    @property
    def dir_size(self):
        
        dir_size = 0
        
        for content in self.contents:
            if isinstance(content, AFile):
                dir_size += content.size
            elif isinstance(content, FileDir):
                dir_size += content.dir_size
                
        return dir_size
                
    def get_child_dir(self, dir_name:str):
        
        for content in self.contents:
            if isinstance(content, FileDir) and content.dir_name == dir_name:
                return content
            
        return None
            
    def check_under_100k(self):
        
        return self.dir_size <= 100_000
    
    def __repr__(self) -> str:
        return f"dir_name:{self.dir_name}, dir_size: {self.dir_size}"



In [22]:

def create_dir_dict(terminal_output: List[str]):

    dir_dict = {}
    current_dir : FileDir = None
    in_list = False

    for idx, line in enumerate(terminal_output):
        command = line.split(" ",2)
        
        if '$' in line:     # Then is a command
            if command[COMMAND] == 'cd':
                if command[ARGUMENT] == "/":
                    in_list = False
                    dir_dict["top"] = FileDir("top", parent=None)
                    current_dir = dir_dict["top"]
                elif command[ARGUMENT] == "..":
                    in_list = False
                    current_dir = dir_dict[current_dir.parent]
                else:
                    in_list = False
                    current_dir = dir_dict[command[ARGUMENT]]
                    
            elif command[COMMAND] == 'ls':
                in_list = True
                continue
        else:
            if command[0] == "dir":
                dir_dict[command[1]] = FileDir(command[1], current_dir.dir_name)
                current_dir.add_contents(dir_dict[command[1]])
            elif command[0].isnumeric():
                current_dir.add_contents(AFile(command[1], int(command[0])))
                
    return dir_dict
                

In [23]:
dir_dict = create_dir_dict(terminal_output)

In [24]:
import numpy as np
dir_100k_list = [dir_.dir_size for dir_ in dir_dict.values() if dir_.check_under_100k()]
np.array(dir_100k_list).sum()

967369

In [25]:
dir_dict["top"].contents

[dir_name:btsgrbd, dir_size: 35167648,
 AFile(name='cprq.fmm', size=3868),
 dir_name:gcbpcf, dir_size: 8024661,
 dir_name:hfm, dir_size: 0,
 AFile(name='lthcng.gnf', size=324644),
 AFile(name='nblfzrb.mrr', size=133181),
 AFile(name='sfrbjmmh.jnj', size=140568),
 dir_name:tfsh, dir_size: 244821,
 dir_name:vlsqgrw, dir_size: 6580859,
 AFile(name='vmpgqbcd', size=202279)]

## ---------------------- TEST ZONE ------------------------

In [9]:
a_string = "cd .."
a_string.split(" ",2)

['cd', '..']